## Combining all downstream used A2TEA data into one RDS object
## Also performing necessary data manipulation

In [ ]:
###packages currently installed via conda

#if (!requireNamespace("BiocManager", quietly = TRUE))
#    install.packages("BiocManager")
#BiocManager::install(version = "3.12")

#BiocManager::install("DESeq2")
#BiocManager::install("ggtree")
#BiocManager::install("Biostrings")

#lfcShrinkage isn't installable at the moment - problem for later
#install.packages("mvtnorm")
#BiocManager::install("apeglm")

#install.packages("data.table")

#install.packages("seqinr")
#install.packages("VennDiagram")
#install.packages("UpSetR")
#install.packages("cowplot")
#install.packages("ggplotify")

###############################################


if (!requireNamespace("BiocManager", quietly=TRUE))
    install.packages("BiocManager")

BiocManager::install("ggtree")
BiocManager::install("ggtreeExtra")
BiocManager::install("Biostrings")

install.packages("VennDiagram")
install.packages("UpSetR")
install.packages("cowplot")
install.packages("ggplotify")

In [1]:
library(DESeq2)
#library(BiocGenerics) # comes with DESeq2
#library(apeglm) # currently difficulties with install # better DESseq2 calc; later 

library(tidyverse)
library(ggtree)
library(Biostrings)
library(seqinr)
library(VennDiagram)
library(UpSetR)
library(cowplot)
library(ggplotify)
library(ape)

Warning message:
“package ‘DESeq2’ was built under R version 4.0.3”
Loading required package: S4Vectors

Warning message:
“package ‘S4Vectors’ was built under R version 4.0.3”
Loading required package: stats4

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rown

#### Find all DESeq2 differential expression RDS objects and load them
##### Names are given based on the species/ecotype/etc. name

In [2]:
dea_list <- list.files(path = "R/deseq2/dea_final/", pattern = "dea_*", full.names=TRUE)
dea_list_short <- list.files(path = "R/deseq2/dea_final/", pattern = "dea_*", full.names=FALSE)

listRDS <- lapply(dea_list, readRDS, .GlobalEnv)

for (i in 1:length(listRDS)){
    assign(str_sub(dea_list_short[[i]], start=5), listRDS[[i]])
}

# remove unused combined list object
rm(listRDS)

In [ ]:
#nice for later - but needs install of package "apeglm"

#link on logFC shrinking
#https://support.bioconductor.org/p/77461/
#?lfcShrink
#lfcShrink(arabidopsis_dea, coef=2, type="apeglm")
#resultsNames(arabidopsis_dea)

In [ ]:
# DESeq2 testing
#At <- readRDS("R/deseq2/dea_final/dea_Arabidopsis_thaliana")
#DESeq2::results(At)
#as.data.frame(results(Arabidopsis_thaliana))

In [3]:
species_list <- vector()

for (i in 1:length(dea_list_short)){
    species_list <- c(species_list, str_sub(dea_list_short[[i]], start=5))
}

list_AllSpeciesDEResultsDataFrames <- list()

for (i in species_list) {
    # individual species file could be deleted but perhaps I'll need them later; kept for now...
    assign(paste0(i, "_DEresultsTable"), as.data.frame(results(get(i))))
    # create gene name vector here (when list is constructed rownames become unique species-gene combination)
    gene <- rownames(get(paste0(i, "_DEresultsTable")))
    df_with_genes <- add_column(get(paste0(i, "_DEresultsTable")), gene, .before = "baseMean")
    # also add a species column which will will also come in handy during the shiny steps
    species <- replicate(nrow(df_with_genes), i)
    df_with_genes <- add_column(df_with_genes, species, .before = "gene")
    # create list of dataframes, which will come in handy
    list_AllSpeciesDEResultsDataFrames[[i]] <- df_with_genes
    # list_AllSpeciesDEResultsDataFrames[[i]] <- (get(paste0(i, "_DEresultsTable")))
}

#str(list_AllSpeciesDEResultsDataFrames)

#create combined "mega" dataframe of all species, which is going  to be used for shiny lookup

combined_AllSpeciesDEResultsDataFrames <- do.call("rbind", list_AllSpeciesDEResultsDataFrames)


#str(combined_AllSpeciesDEResultsDataFrames)
#head(combined_AllSpeciesDEResultsDataFrames)
#head(list_AllSpeciesDEResultsDataFrames[[1]])
#?rbind

### creating long format HOG-genes relation table

In [4]:
HOG_file_raw <- read_delim("orthofinder/final-results/Phylogenetic_Hierarchical_Orthogroups/N0.tsv", delim = "\t")

#drop the OG, Gene Tree Parent Clade
HOG_file_short <- HOG_file_raw %>% select (-c(OG, `Gene Tree Parent Clade`)) 


#merge all species columns into one and remove the solo ones
#use all colnames except first - thus all species
HOG_file_merged <- HOG_file_short %>% unite(x,
                                          c(colnames(HOG_file_short)[-1]),
                                          sep = ",", 
                                          na.rm = TRUE,
                                          remove = TRUE)

#transform into long format - each row 1 gene and it's corresponding HOG in seperate columns
HOG_file_long <- HOG_file_merged %>%
                    mutate(unpacked = str_split(x, ",")) %>%
                    unnest(cols = c(unpacked)) %>%
                    mutate(genes = str_trim(unpacked)) %>% 
                    select(-c(x, unpacked)) %>% 
                    rename(gene = genes)

str(HOG_file_long)


── Column specification ──────────────────────────────────────────────────────────────
cols(
  HOG = col_character(),
  OG = col_character(),
  `Gene Tree Parent Clade` = col_character(),
  Arabidopsis_thaliana = col_character(),
  Hordeum_vulgare = col_character(),
  Zea_mays = col_character()
)




tibble [91,546 × 2] (S3: tbl_df/tbl/data.frame)
 $ HOG : chr [1:91546] "N0.HOG0000000" "N0.HOG0000000" "N0.HOG0000000" "N0.HOG0000000" ...
 $ gene: chr [1:91546] "Zm00001d049811" "Zm00001d010350" "Zm00001d012872" "Zm00001d023233" ...


### adding specific HOG or singleton info as HOG column to DE tables

In [5]:
#?full_join
HOG_DE.a2tea <- full_join(combined_AllSpeciesDEResultsDataFrames, HOG_file_long, 
              by = c("gene")) %>% 
                replace_na(list(HOG = "singleton"))

#add column for significance - default level is padj <= 0.1

#subset tidyverse with the right functions ;D
#filter(HOG_DE.a2tea, padj < 0.05)
#?drop_na

significant <- vector()
for (FDR in HOG_DE.a2tea$padj) {
    if (!is.na(FDR) && FDR < 0.1) {
        significant <- c(significant, "yes")
    }
    else if (!is.na(FDR) && FDR > 0.1) {
        significant <- c(significant, "no")
    }
    else if (is.na(FDR)) {
        significant <- c(significant, "no")
    }
}

HOG_DE.a2tea <- add_column(HOG_DE.a2tea, significant, .after = "padj")

In [ ]:
#?full_join

### load in the rest of the data - hypotheses, trees, fasta and msa for the start 
### + toDo general stats, especially once Orthofinder calculates for HOG)
### +toDo Venn DIagrammes (until 5 species or so then alternative plots)

In [26]:
#extended_BLAST_hits <- readRDS("tea/2/extended_BLAST_hits/extended_BLAST_hits.RDS")
setClass("extended_BLAST_hits", 
         slots=list(blast_table="tbl_df",
                    nrow_table="numeric",
                    num_genes_HOG="numeric",
                    num_genes_extend="numeric",
                    num_genes_complete="numeric",
                    genes_HOG="tbl_df")
         )
#expanded_OGs_short
extended_BLAST_hits[["N0.HOG0000207"]]@nrow_table

[1] 441

In [30]:
hypotheses <- read_delim("hypotheses.tsv", delim = "\t")


#create hypotheses object
# each object has list of exp. OGs
#iterate over list of hyptheses and associate expansions, fasta, msa and trees


#first goal - hypothesis object with hypothesis and all exp.HOGs

#could do it over objects in tea directory
#list.files(path = "tea", pattern = "*", full.names=FALSE)


#alternatively via parsing of hyptheses.tsv
hypotheses$hypothesis


## define three classes
# class for the expanded_OG - containing all different types of data we have on it
setClass("expanded_OG", slots=list(genes="spec_tbl_df",
                                   blast_table="tbl_df",
                                   nrow_table="numeric",
                                   num_genes_HOG="numeric",
                                   num_genes_extend="numeric",
                                   num_genes_complete="numeric",
                                   genes_HOG="tbl_df",
                                   fasta_files="list", 
                                   msa="AAMultipleAlignment", 
                                   tree="phylo"))


# class for the hypotheses
# adding a prototype is essential here for ortho_intersect_plot since gg has to be defined as 
# register an old-style S3 class using setOldClass
#https://stackoverflow.com/questions/12636056/why-sometimes-i-cant-set-a-class-definition-as-slot-in-a-s4-class
#https://stackoverflow.com/questions/13841400/use-s3-virtual-class-as-slot-of-an-s4-class-got-error-got-class-s4-should-b
#had trouble getting this to work; thus solved differently
# -> seperate list object "Ortho_intersect_plots" hypothesis specific access via index
setClass("hypothesis", 
#         prototype=prototype(ortho_intersect_plot=structure(list(), class="gList")),
         slots=list(description="character", 
                                  number="character",
                                  expanded_in ="character", 
                                  compared_to="character", 
                                  expanded_OGs="list",
                                  species_tree="phylo"))
#                                  ortho_intersect_plot="gg"))

# class for extended BLAST hits info
setClass("extended_BLAST_hits", 
         slots=list(blast_table="tbl_df",
                    nrow_table="numeric",
                    num_genes_HOG="numeric",
                    num_genes_extend="numeric",
                    num_genes_complete="numeric",
                    genes_HOG="tbl_df")
         )


rm(t)
rm(hypothesis_1)
rm(hypothesis_2)
rm(HYPOTHESES.a2tea)


#remove protein_names in the snakemake pipeline - directories clean enough


for (hypothesis in hypotheses$hypothesis) {
    # read-in extended_BLAST_hits.RDS object of hypothesis
    extended_BLAST_hits <- readRDS(paste0("tea/", hypothesis, "/extended_BLAST_hits/extended_BLAST_hits.RDS"))
    
    # print(paste0("hypothesis_", hypo))
    
    # create empty list object for hypothesis #
    assign(paste0("hypothesis_", hypothesis), list())

    # assign list of names
    expanded_OGs <- list.files(path = paste0("tea/", hypothesis, "/exp_OGs_proteinnames/"), 
                               pattern = "*", 
                               full.names=FALSE)
    expanded_OGs_short <- str_sub(expanded_OGs, end=-5)

    # assign or subset the orthofinder species_tree based on the current hypothesis
    speciesTree <- ape::keep.tip(read.tree("orthofinder/final-results/Species_Tree/SpeciesTree_rooted_node_labels.txt"), 
                                 c(unlist(str_split(hypotheses$expanded_in[hypothesis], ";")), 
                                   unlist(str_split(hypotheses$compared_to[hypothesis], ";"))))
    
    
    #create empty list
    t <- list()

    for (exp_OG in expanded_OGs_short) {
    # print(exp_OG)
        test <- new("expanded_OG", 
             genes=read_table(paste0("tea/", hypothesis, "/exp_OGs_proteinnames/", exp_OG, ".txt"), col_names = FALSE),
             blast_table=extended_BLAST_hits[[exp_OG]]@blast_table,
             num_genes_HOG=extended_BLAST_hits[[exp_OG]]@num_genes_HOG,
             num_genes_extend=extended_BLAST_hits[[exp_OG]]@num_genes_extend,
             num_genes_complete=extended_BLAST_hits[[exp_OG]]@num_genes_complete,
             genes_HOG=extended_BLAST_hits[[exp_OG]]@genes_HOG,
             fasta_files=read.fasta(paste0("tea/", hypothesis, "/fa_records/", exp_OG,".fa"), seqtype = "AA", as.string = TRUE), 
             msa=readAAMultipleAlignment(paste0("tea/", hypothesis, "/muscle/", exp_OG, ".afa")), 
             tree=read.tree(paste0("tea/", hypothesis, "/trees/", exp_OG, ".tree")))
   
        x <- list(test)
        names(x) <- paste0(exp_OG)
        t <- c(t, x)
    }
 
    # create for each hypothesis a complete hypothesis object with correct naming
    # order: hypothesis@info (name, number, expand, compared, exp OGs)$indiv. exp_OGs@objects (exp.OG list, fa, msa, tree)
    # "expanded_in" and "compared_to" are split on ";" in case their size is > 1
    assign(paste0("hypothesis_", hypothesis),
              new("hypothesis", 
                  description=hypotheses$name[hypothesis],
                  number=as.character(hypothesis),
                  expanded_in=unlist(str_split(hypotheses$expanded_in[hypothesis], ";")),
                  compared_to=unlist(str_split(hypotheses$compared_to[hypothesis], ";")), 
                  expanded_OGs=t,
                  species_tree=speciesTree))#,
#                  ortho_intersect_plot=NULL))
}


# create empty list for final complete hypothesis object

#rm

# create name list of all hypotheses
hypotheses_list <- ls(pattern = "hypothesis_")

# create empty list object to completely hold all hypotheses and associated data
HYPOTHESES.a2tea <- list()

for (hypothesis in hypotheses_list) {
    h <- list(get(hypothesis))
    names(h) <- paste0(hypothesis)
    HYPOTHESES.a2tea <- c(HYPOTHESES.a2tea, h)
}

rm(hypothesis_1)
rm(hypothesis_2)


# final object is called:
# HYPOTHESES.a2tea


#add expanded OGs as column
#    add_column(get(paste0("hypothesis_", hypo)), expanded_OGs)
#class(hypothesis_1@expanded_OGs) <- c("expanded_OG", class(hypothesis_1@expanded_OGs))



── Column specification ──────────────────────────────────────────────────────────────
cols(
  hypothesis = col_double(),
  name = col_character(),
  expanded_in = col_character(),
  compared_to = col_character()
)




[1] 1 2

Warning message in rm(hypothesis_2):
“object 'hypothesis_2' not found”
Warning message in rm(HYPOTHESES.a2tea):
“object 'HYPOTHESES.a2tea' not found”

── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



──


── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ───────


── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ───────

...


── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ───────


── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ───────


── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ───────


── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ───────


── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ──────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)



── Column specification ───────

In [41]:
length(hypotheses$hypothesis)
length(HYPOTHESES.a2tea$hypothesis_2@expanded_OGs$N0.HOG0000027@genes$X1)
length(HYPOTHESES.a2tea$hypothesis_2@expanded_OGs$N0.HOG0000027@fasta_files)
HYPOTHESES.a2tea$hypothesis_2@expanded_OGs$N0.HOG0000028@blast_table
#?setClass

[1] 2

[1] 122

[1] 121

qseqid,qseqid_name,sseqid,sseqid_name,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2_2925,Zm00001d001920,2_2925,Zm00001d001920,100.0,777,0,0,1,777,1,777,0,1562
2_2925,Zm00001d001920,2_36827,Zm00001d053905,98.5,777,12,0,1,777,914,1690,0,1539
2_2925,Zm00001d001920,2_25035,Zm00001d047609,97.9,777,16,0,1,777,1059,1835,0,1533
2_2925,Zm00001d001920,2_6835,Zm00001d044057,98.3,773,13,0,1,773,1027,1799,0,1532
2_2925,Zm00001d001920,2_5215,Zm00001d040584,98.3,773,13,0,1,773,1027,1799,0,1530
2_2925,Zm00001d001920,2_6936,Zm00001d053790,98.2,773,14,0,1,773,993,1765,0,1530
2_2925,Zm00001d001920,2_27207,Zm00001d023474,97.0,774,23,0,1,774,987,1760,0,1513
2_2925,Zm00001d001920,2_37707,Zm00001d038010,97.0,773,23,0,1,773,1034,1806,0,1512
2_2925,Zm00001d001920,2_5901,Zm00001d038953,96.5,777,27,0,1,777,501,1277,0,1510


### HOG level tables -needs to be behind creation of hypotheses, because they are used here

In [ ]:
# create hypothesis specfic HOG level file

#find all tibbles -list
#list.files(path = "tea/1/expansion_tibble", pattern = "expansion_tibble.rds", full.names=TRUE)


#length(hypotheses_list)


HOG_level_list <- list()

#for (i in 1:length(hypotheses_list)) {
for (i in 1:length(hypotheses$hypothesis)) {
    assign(paste0("hypothesis_", i, "_expansion_tibble"),
           readRDS(
               list.files(path = paste0("tea/", i, "/expansion_tibble"), 
                      pattern = "expansion_tibble.rds", 
                      full.names=TRUE)
                  )
           )
    # add to list and name list entries according to hypothesis
    hog_level <- list(get(paste0("hypothesis_", i, "_expansion_tibble")))
    names(hog_level) <- paste0("hypothesis_", i)
    HOG_level_list <- c(HOG_level_list, hog_level)
}

#e.g. the following displays the tibble
#hypothesis_2_expansion_tibble

#r
#append "_gene_count" to all species in HOG tables
#get number of significantly regulated genes per HOG (per species and total) 
# -> (hmm perhaps do this once and then merge?)


# get count of species, HOG, significant combination from HOG_DE.a2tea
# reduce it to HOG, species, count
sig_genes_per_species_and_HOG <- HOG_DE.a2tea %>% 
                                     filter(significant == c("yes")) %>%
                                     filter(HOG != c("singleton")) %>%
                                        group_by(HOG, species, significant) %>% 
                                        mutate(count = n()) %>%
                                        ungroup() %>%
                                        select(HOG, species, count) %>%
            # https://www.r-bloggers.com/2018/05/workaround-for-tidyrspread-with-duplicate-row-identifiers/
            # spread error when no indexing for data
                                            group_by(species) %>% 
                                            mutate(grouped_id = row_number()) %>%  
                                            spread(species, count) %>% 
                                            select(-grouped_id) %>%
            # easy workaround for duplicated rows
                                                distinct()%>% 
            # rename species columns containing now the counts of sig. DE genes
                                                    rename_at(vars(-HOG), ~ paste0(., '_sigDE'))
    # and also add a column summing the rowwise sig. DE counts for all species
    total_sigDE <- sig_genes_per_species_and_HOG %>% select(-HOG) %>% rowSums(na.rm = TRUE) 
    sig_genes_per_species_and_HOG <- add_column(sig_genes_per_species_and_HOG, total_sigDE)
    sig_genes_per_species_and_HOG <- sig_genes_per_species_and_HOG  %>%
                                         group_by(HOG) %>%
                                         # mutate all NAs to 0s
                                         mutate_at(vars(-group_cols()), ~replace(., is.na(.), 0)) %>%
                                         # merge rows per HOG - results in one line per HOG
                                         #summarise_all(funs(sum))
                                         summarise_all(list(sum))

# change the zeros back to NAs
    sig_genes_per_species_and_HOG <- sig_genes_per_species_and_HOG  %>%
                                         na_if(0)

#sig_genes_per_species_and_HOG
#total_sigDE
#inner_join(HOG_level_list[[1]], sig_genes_per_species_and_HOG, by = c("HOG"))
#full_join important - better to keep all HOG rows and allow filtering dynamically in the WebAPP itself

for (i in 1:length(HOG_level_list)) {
    HOG_level_list[[i]] <- HOG_level_list[[i]] %>% rename_at(vars(-HOG, -expansion), ~ paste0(., '_total'))
    HOG_level_list[[i]] <- full_join(HOG_level_list[[i]], sig_genes_per_species_and_HOG, by = c("HOG"))
}

str(HOG_level_list)


#head(HOG_level_list)

#df %>% rename_at(vars(-Name, -State), ~ paste0(., '_2017'))


In [ ]:
summary(HYPOTHESES.a2tea)

In [ ]:
HOG_level_list$hypothesis_1
HOG_level_list$hypothesis_1 %>%
    filter(HOG == "N0.HOG0000318")

#HOG_level_list$hypothesis_1 %>% group_by(HOG) %>% summarise(sum(total_sigDE))

### creating Venn (or alternative diagrammes for the hypothesis)

In [ ]:
# create this awesome function to change area of plotting
# https://stackoverflow.com/questions/1279003/specify-width-and-height-of-plot
set_plot_dimensions <- function(width_choice, height_choice) {
        options(repr.plot.width=width_choice, repr.plot.height=height_choice)
        }
# use funtion to set area a bit wider
set_plot_dimensions(32, 10)



#step 1: removing all species not in hypothesis or keep only those that are
# using HOG_level_list and looking at the "_total" columns
#  only species in this hypothesis are in here ;D
# -> still drop all columns not necessary?
#step 2: perform check on size of this set
# -> if < 5 then create Venn diagramme
# -> if > 5 then create alternative plot
# create plot and safe as sub element in HYPOTHESES.a2tea object 



#HOG_DE.a2tea
#HOG_level_list[[1]]
#choose based on set length 2,3,4,5 after that UpsetR


Ortho_intersect_plots <- list()

for (i in 1:length(HOG_level_list)) {

workset <-  HOG_level_list[[i]]
# drop columns that are unnecessary
workset <- workset %>% select(HOG, ends_with("_total")) 
# add this point we can easily compute the number of species/ecotypes etc. 
# #species = (columns of current workset - 1) 
species_num <- ncol(workset) - 1
# change 0 to NAs since we can more easily filter on that
is.na(workset) <- workset==0
# remove '_total' from column names
#str_remove(colnames(workset), "_total")
workset <- workset %>% setNames(names(workset) %>% stringr::str_replace("_total",""))

# for testing purposes - remove Genotype vectors
rm(Genotype1,Genotype2,Genotype3)

genotypeList <- list()
for (j in 1:species_num) {
    assign(paste0("Genotype", j), as.vector(simplify_all(workset[!is.na(workset[j+1]),"HOG"])[[1]]))
    subList <- list(get(paste0("Genotype", j)))
    names(subList) <- colnames(workset)[j+1]
    genotypeList <- c(genotypeList, subList)
}


#using UpSetR
upsetOrtho <- upset(fromList(genotypeList), 
                   order.by = "freq",
                   empty.intersections = "on",
                   point.size = 3.5, 
                   line.size = 2, 
                   mainbar.y.label = "HOG Intersections", 
                   text.scale = c(2, 1.3, 1, 1, 2, 2))

p <- upsetOrtho

# if we have less than 5 species draw a venn diagramme
# combine both plots with some trickery
if (species_num < 5) {
    
# draw venn diagramme
orthoVenn <- venn.diagram(genotypeList,
                           print.mode = c("raw","percent"),
                           sigdigs = 1,
                           filename = NULL,
                           category = names(genotypeList),
                           fill = rainbow(length(names(genotypeList))),
                           cat.col = rainbow(length(names(genotypeList))),
                           cat.cex = 2,
                           cex = 1.5,
                           margin = 0.05
                           )    
    
# as.grob from ggplotify is necessary
orthoIntersect <- plot_grid(orthoVenn, 
          as.grob(upsetOrtho), 
          rel_widths = c(1/3, 2/3))

#p <- ggplotify::as.ggplot(orthoIntersect)
p <- orthoIntersect
    }

Ortho_intersect_plots[[i]] <- p

}
    
    
    


In [ ]:
Ortho_intersect_plots[[1]]

### computation of tea value (first idea)

expansion ratio (exp/com) (9/4) x expression ratio (com/exp) (2/6) 
---- don't need (n_species_expanded/n_species_compared); stays equal in a hypothesis; similar to p-values not          comparable between different experiments
divided by HOG size (all genes in HOG for this particular hypothesis) x number of significantly regulated genes


problem - how to deal with missing values???

In [ ]:
# should I only have hypothesis species (total count and sigDE) HOG_level_list tables?

#new column tea-value
#if expansion yes continue; else NA value
#because we set expansion as criterium, species gene count has to be >=1
#sig DE counts can be NA ! - we could just add a pseudocount of 1 at these positions for the calculation

#so per line that has expansion = yes:
#get all expanded species; sum their gene counts; sum their sigDE counts
#get all compared species; sum their gene counts; sum their sigDE counts
#expansion_ratio <- divide summed gene count expanded by summed gene count compared
#expression_ratio <- divide summed SigDE count expanded by summed SigDE count compared
#e2_ratio <- divide expansion_ratio by expression_ratio
#tea-value <- divide e2_ratio by total number of genes in HOG and then by total number of SigDE in HOG


# outer most loop; going through hypotheses in HOG_level_list
for (i in 1:length(HOG_level_list)) {
    
    # access the names of all species important for the hypothesis - expanded_in; compared_to
    print(c(unlist(str_split(HYPOTHESES.a2tea[[i]]@expanded_in, ";")), 
            unlist(str_split(HYPOTHESES.a2tea[[i]]@compared_to, ";"))))
    
    # creating a relevant species vector might make this more future proof;
    # since I might change the completeness of the HOG_level_list elements
#    relevant_species <- c(unlist(str_split(HYPOTHESES.a2tea[[i]]@expanded_in, ";")), 
#                          unlist(str_split(HYPOTHESES.a2tea[[i]]@compared_to, ";")))
    
    expanded_species <- unlist(str_split(HYPOTHESES.a2tea[[i]]@expanded_in, ";"))
    
    compared_species <- unlist(str_split(HYPOTHESES.a2tea[[i]]@compared_to, ";"))                              
                                  
    ## pre-allocate empty vector for tea value with length of nrow of current hypothesis of HOG_level_list
    # neat; because NAs already in there so I can just skip if the HOG shows no expansion ;D
    tea_value <- rep(NA, nrow(HOG_level_list[[i]]))
    
    
    ## create "geneCount_workset" containing HOGs and counts of all relevant species
    geneCount_workset <-  HOG_level_list[[i]]

    # create geneCount_EXP_workset & geneCount_COMP_workset + drop columns that are unnecessary                  
    geneCount_EXP_workset <- geneCount_workset %>% 
                             select(ends_with("_total")) %>%
                             select(contains(expanded_species))
                        # also remove the "_total" ending
    geneCount_EXP_workset <- geneCount_EXP_workset %>% 
                             setNames(names(geneCount_EXP_workset) %>% 
                             stringr::str_replace("_total",""))
                          
    geneCount_COMP_workset <- geneCount_workset %>% 
                             select(ends_with("_total")) %>%
                             select(contains(compared_species))
                        # also remove the "_total" ending
    geneCount_COMP_workset <- geneCount_COMP_workset %>% 
                             setNames(names(geneCount_COMP_workset) %>% 
                             stringr::str_replace("_total",""))
                    

    ## create "geneSig_workset" containing HOGs and counts of all relevant species 
    geneSig_workset <-  HOG_level_list[[i]]
    
    # create geneSig_EXP_workset & geneSig_COMP_workset + drop columns that are unnecessary                  
    geneSig_EXP_workset <- geneSig_workset %>% 
                             select(ends_with("_sigDE")) %>%
                             select(contains(expanded_species))
                        # also remove the "_sigDE" ending
    geneSig_EXP_workset <- geneSig_EXP_workset %>% 
                             setNames(names(geneSig_EXP_workset) %>% 
                             stringr::str_replace("_sigDE",""))
                          
    geneSig_COMP_workset <- geneSig_workset %>% 
                             select(ends_with("_sigDE")) %>%
                             select(contains(compared_species))
                        # also remove the "_sigDE" ending
    geneSig_COMP_workset <- geneSig_COMP_workset %>% 
                             setNames(names(geneSig_COMP_workset) %>% 
                             stringr::str_replace("_sigDE",""))

                               
    # nested loop that goes through current hypothesis HOG_level_list line by line
    for (j in 1:nrow(HOG_level_list[[i]])) {
        # write value into index slot if current HOG does pass hard filter expansion criterium
        # if expansion == "no" do nothing; since we prefilled all positions with NA
        if (HOG_level_list[[i]][j,"expansion"] == "yes") {
            
            ## compute all the different sub-values for the tea_value
            # count of all genes in expanded species in HOG
            geneCount_EXP_HOG <- geneCount_EXP_workset[j,] %>%
                                 dplyr::rowwise() %>%
                                 sum()
            
            # count of all genes in compared species in HOG
            geneCount_COMP_HOG <- geneCount_COMP_workset[j,] %>%
                                      dplyr::rowwise() %>%
                                      sum()
            
            # sum genes of expanded & compared species in HOG
            geneCount_BOTH_HOG <- sum(geneCount_EXP_HOG, geneCount_COMP_HOG)
            
            
            # count of all sig DE genes in expanded species in HOG
            geneSig_EXP_HOG <- geneSig_EXP_workset[j,] %>%
                                 dplyr::rowwise() %>%
                                 sum()
            
            # count of all sig DE genes in compared species in HOG
            geneSig_COMP_HOG <- geneSig_COMP_workset[j,] %>%
                                      dplyr::rowwise() %>%
                                      sum()
            
            ## sigDE columns can contain zero (NA in table) counts
            ## we exchange this for a pseudocount 1
            ## doesn't impact tea_value too much
            if (is.na(geneSig_EXP_HOG)) {
                geneSig_EXP_HOG <- 1
            }
            
            if (is.na(geneSig_COMP_HOG)) {
                geneSig_COMP_HOG <- 1
            }
            
            # sum Sig DE genes of expanded & compared species in HOG
            geneSig_BOTH_HOG <- sum(geneSig_EXP_HOG, geneSig_COMP_HOG)

            
            ## perform tea value calculation
            int_tea_value <- (geneCount_EXP_HOG / geneCount_COMP_HOG) * (geneSig_COMP_HOG / geneSig_EXP_HOG) / 
                                                  (geneCount_BOTH_HOG * geneSig_BOTH_HOG)
            
            # lastly, add computed value to tea_value at index of current j loop position
            tea_value[j] <- int_tea_value
        }
    }
    
    ## add tea_value as new column to HOG_level_list table
    HOG_level_list[[i]] <- HOG_level_list[[i]] %>% 
                               add_column(tea_value, .after = "HOG")
    
}

In [ ]:
HOG_level_list$hypothesis_1 %>%
    arrange(tea_value)

In [ ]:
#c(unlist(str_split(HYPOTHESES.a2tea[[2]]@expanded_in, ";")), 
#  unlist(str_split(HYPOTHESES.a2tea[[2]]@compared_to, ";")))
#geneSig_workset <-  HOG_level_list[[1]]
#names(geneSig_workset)
#head(geneSig_workset %>% setNames(names(geneSig_workset) %>% stringr::str_replace("_sigDE","")))
#HOG_level_list[[i]][1,"expansion"]
#tea_value[2] <- "asdasdd"
str(tea_value)
head(geneCount_EXP_workset[2,])

HOG_level_list[[1]][3,c(2,3)] %>%
    dplyr::rowwise() %>%
    sum()

In [ ]:
summary(HYPOTHESES.a2tea)
#HOG_level_list[[2]] %>% add_column(tea_value, .after = "HOG")

In [ ]:
HYPOTHESES.a2tea$hypothesis_1@expanded_OGs$N0.HOG0000234@genes

### Last step: saving everything to one file which is input for the A2TEA WebApp

In [ ]:
save(hypotheses, 
     HYPOTHESES.a2tea, 
     HOG_DE.a2tea, 
     HOG_level_list, 
     Ortho_intersect_plots,
     file = "tea/A2TEA_finished.RData")

# To load the data again
#load("tea/A2TEA_finished.RData")

#perhaps use this to name output based on time?
#paste0("tea/A2TEA_finished_",
#    str_replace_all(
#    str_replace_all(
#        Sys.time(), ":", "-"),
#    " ", "_"
#                    ), ".RData")

In [43]:
final <- load("tea/A2TEA_finished.RData")

In [49]:
HYPOTHESES.a2tea$hypothesis_1@expanded_OGs$N0.HOG0000207@blast_table

qseqid,qseqid_name,sseqid,sseqid_name,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1_20479,HORVU4Hr1G080730,1_20479,HORVU4Hr1G080730,100.0,545,0,0,1,545,1,545,0.00e+00,1103
2_38607,Zm00001d027549,2_38607,Zm00001d027549,100.0,543,0,0,1,543,1,543,0.00e+00,1083
0_11343,AT5G43340,0_11343,AT5G43340,100.0,516,0,0,1,516,1,516,0.00e+00,1007
1_20479,HORVU4Hr1G080730,2_38607,Zm00001d027549,82.5,538,82,3,1,526,1,538,0.00e+00,880
2_38607,Zm00001d027549,1_20479,HORVU4Hr1G080730,82.5,538,82,3,1,538,1,526,0.00e+00,876
2_38607,Zm00001d027549,2_18960,Zm00001d031875,77.0,539,114,3,3,541,1,529,1.94e-302,828
1_20479,HORVU4Hr1G080730,2_18960,Zm00001d031875,77.8,527,111,2,3,525,1,525,2.10e-302,828
2_18960,Zm00001d031875,2_38607,Zm00001d027549,77.0,539,114,3,1,529,3,541,1.07e-299,821
2_18960,Zm00001d031875,1_20479,HORVU4Hr1G080730,77.8,527,111,2,1,525,3,525,1.12e-298,818
